# 고밀도 벡터
- 머신 러닝과 데이터 분석에 널리 사용되는 수치 데이터 표현
- 복잡한 패턴과 관계를 효과적으로 포착할 수 있어 고차원 공간에서 데이터를 더 쉽게 분석하고 처리할 수 있음
- 특정 애플리케이션과 요구 사항에 따라 수십 개에서 수백 개 또는 수천 개에 이르는 고정된 수의 차원을 갖음

### 핵심개념
- 다차원 표현: 각 점은 밀버스, 벡터 데이터베이스, 검색 시스템 등과 같은 개념적 객체를 나타내며, 그 위치는 해당 치수의 값에 따라 결정됨
- 의미관계: 점 사이의 거리는 개념 간의 의미적 유사성을 반영, 점이 가까울수록 의미적으로 더 연관성이 높은 개념을 나타냄
- 클러스터링 효과: 밀버스, 벡터 데이터베이스, 검색 시스템 등 관련 개념이 공간에서 서로 가깝게 배치되어 의미론적 클러스터를 형성

- 고밀도 벡터는 이미지용 CNN 모델(예: ResNet, VGG), 텍스트용 언어 모델(예: BERT, Word2Vec) 등 다양한 임베딩 모델을 사용하여 생성할 수 있음
- 원시 데이터를 고차원 공간의 점으로 변환하여 데이터의 의미적 특징을 포착함

# 고밀도 벡터 사용
## 벡터 필드 추가
- datatype을 지원되는 고밀도 벡터 데이터 유형으로 설정, 지원되는 고밀도 벡터 데이터 유형은 데이터 유형을 참조
- dim 매개변수를 사용하여 고밀도 베겉의 차원을 지정

### 지원하는 벡터 유형
- FLOAT_VECTOR
    - 32비트 부동 소수점 숫자를 저장, 유사한 벡터를 구별하는 등 높은 정밀도가 필요한 시나리오에 이상적
- FLOAT16_VECTOR
    - 딥러닝 및 GPU 계산에 사용되는 16비트 반정밀도 부동 소수점 숫자를 저장, 추천 시스템의 저정밀 리콜 단계와 같이 정밀도가 덜 중요한 시나리오에서 저장 공간을 절약할 수 있음
- BFLOAT16_VECTOR
    - 16비트 뇌 부동 소수점(bfloat16) 숫자를 저장하여 Float32와 동일한 범위의 지수를 제공하지만 정밀도는 떨어짐, 대규모 이미지 검색과 같이 대량의 벡터를 빠르게 처리해야하는 시나리오에 적합
- INT8_VECTOR
    - 8비트 정수(int8)이고 각 요소의 범위가 -128 ~ 127 사이인 벡터를 저장, HNSW 인덱스만 지원

In [8]:
from pymilvus import MilvusClient, DataType

client = MilvusClient(uri="http://localhost:19530")

schema = client.create_schema(
    auto_id=True,
    enable_dynamic_fields=True,
)

schema.add_field(field_name="pk", datatype=DataType.VARCHAR, is_primary=True, max_length=100)
schema.add_field(field_name="dense_vector", datatype=DataType.FLOAT_VECTOR, dim=4)

{'auto_id': True, 'description': '', 'fields': [{'name': 'pk', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 100}, 'is_primary': True, 'auto_id': False}, {'name': 'dense_vector', 'description': '', 'type': <DataType.FLOAT_VECTOR: 101>, 'params': {'dim': 4}}], 'enable_dynamic_field': False}

# 벡터 필드에 대한 인덱스 매개변수 설정
- 시맨틱 검색을 가속화하려면 벡터 필드에 대한 인덱스를 생성해야함, 인덱싱은 대규모 벡터 데이터의 검색 효율성을 크게 향상시킬 수 있음
- AUTOINDEX 유형을 사용하여 dense_vector 필드에 대해 dense_vector_index라는 이름의 인덱스가 생성
- metric_type은 IP로 설정되어 내적 곱이 거리 메트릭으로 사용됨

In [11]:
# 인덱스 파라미터 설정 - AUTOINDEX ❌ → IVF_FLAT or HNSW 등으로 변경
index_params = client.prepare_index_params()
index_params.add_index(
    field_name="dense_vector",
    index_name="dense_vector_index",
    index_type="IVF_FLAT",  # ✅ AUTOINDEX → IVF_FLAT, HNSW, FLAT 등으로 변경
    metric_type="IP",       # 또는 L2, COSINE
    params={"nlist": 128}   # IVF 계열은 반드시 nlist 필요
)


## 컬렉션 만들기
- 밀도 벡터 및 인덱스 매개변수 설정이 완료되면 밀도 벡터를 포함하는 컬렉션을 만들 수 있음
- create_collection 메서드를 사용하여 my_collection 이라는 이름의 컬렉션을 생성

In [ ]:
if client.has_collection(collection_name="my_collection"):
    client.drop_collection(collection_name="my_collection")

client.create_collection(
    collection_name="my_collection",
    schema=schema,
    index_params=index_params
)

## 데이터 삽입
- 컬렉션을 만든 후 insert 메서드를 사용하여 고밀도 벡터가 포함된 데이터를 추가, 삽입되는 고밀도 벡터의 차원이 고밀도 벡터 필드를 추가할 때 정의한 dim값과 일치하는지 확인

In [15]:
data = [
    {"dense_vector": [0.1, 0.2, 0.3, 0.4]},
    {"dense_vector": [0.5, 0.6, 0.7, 0.8]},
]

client.insert(
    collection_name="my_collection",
    data=data
)

{'insert_count': 2, 'ids': ['459127794345938947', '459127794345938948'], 'cost': 0}

# 유사도 검색 수행
- 밀도 벡터 기반의 시맨틱 검색은 밀버스의 핵심 기능 중 하나로, 벡터 사이의 거리를 기반으로 쿼리 벡터와 가장 유사한 데이터를 빠르게 찾을 수 있음
- 유사도 검색을 수행하려면 쿼리 벡터와 검색 파라미터를 준비한 다음 search 메서드를 호출

In [20]:
search_params = {
    "params": {"nprobe": 10},
}

query_vecotr = [0.1, 0.2, 0.3, 0.7]

res = client.search(
    collection_name="my_collection",
    data=[query_vecotr],
    anns_field="dense_vector",
    search_params=search_params,
    limit=5,
    output_fields=["pk"],
)

In [22]:
print(res)

data: [[{'pk': '459127794345938948', 'distance': 0.940000057220459, 'entity': {'pk': '459127794345938948'}}, {'pk': '459127794345938947', 'distance': 0.41999998688697815, 'entity': {'pk': '459127794345938947'}}]]
